# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bardai,11.6500,7.2333,21.76,93,96,3.17,NG,1724620279
1,1,hermanus,-34.4187,19.2345,12.27,70,89,5.87,ZA,1724620347
2,2,udachny,66.4167,112.4000,7.07,89,65,2.50,RU,1724620349
3,3,invercargill,-46.4000,168.3500,3.34,98,100,1.34,NZ,1724620350
4,4,bahia de kino,28.8333,-111.9333,33.80,39,3,4.35,MX,1724620351


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    tiles = "OSM"
)

# Display the map
map_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
# criteria: max temp lower than 27 degrees, higher than 21
    # wind speed less than 4.5 m/s
    # zero cloudiness
ideal_city_na_df = city_data_df[(city_data_df["Max Temp"]<27) & 
                             (city_data_df["Max Temp"]>21) & 
                             (city_data_df["Wind Speed"]<4.5) & 
                             (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_city_df = ideal_city_na_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
65,65,dingli,35.8614,14.3822,25.09,83,0,3.60,MT,1724620428
123,123,mastung,29.7994,66.8450,25.00,44,0,1.18,PK,1724620499
128,128,sirjan,29.4520,55.6814,24.39,16,0,3.77,IR,1724620505
165,165,rodolfo sanchez taboada,31.7167,-116.5667,24.50,49,0,1.79,MX,1724620553
180,180,gaigeturi,33.4644,126.3183,25.03,78,0,2.06,KR,1724620571
248,248,kone,-21.0595,164.8658,22.69,69,0,2.39,NC,1724620660
302,302,jamestown,42.0970,-79.2353,26.80,47,0,4.12,US,1724620726
388,388,tripoli,32.8752,13.1875,26.83,70,0,3.29,LY,1724620841
449,449,salmas,38.1973,44.7653,22.70,31,0,2.42,IR,1724620918
481,481,pacific grove,36.6177,-121.9166,25.82,72,0,3.09,US,1724620958


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_columns = ["City", "Country", "Lat","Lng", "Humidity"]
hotel_df = ideal_city_df[hotels_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
65,dingli,MT,35.8614,14.3822,83,
123,mastung,PK,29.7994,66.8450,44,
128,sirjan,IR,29.4520,55.6814,16,
165,rodolfo sanchez taboada,MX,31.7167,-116.5667,49,
180,gaigeturi,KR,33.4644,126.3183,78,
248,kone,NC,-21.0595,164.8658,69,
302,jamestown,US,42.0970,-79.2353,47,
388,tripoli,LY,32.8752,13.1875,70,
449,salmas,IR,38.1973,44.7653,31,
481,pacific grove,US,36.6177,-121.9166,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
language = "en"

params = {"categories":categories,
          "radius":radius,
          "language":language,
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dingli - nearest hotel: Maple farm B&B
mastung - nearest hotel: No hotel found
sirjan - nearest hotel: خانه معلم سیرجان
rodolfo sanchez taboada - nearest hotel: Estero Beach Resort
gaigeturi - nearest hotel: 제주 토비스콘도
kone - nearest hotel: Pacifik Appartels
jamestown - nearest hotel: DoubleTree Jamestown
tripoli - nearest hotel: الاصيل
salmas - nearest hotel: هتل آتا
pacific grove - nearest hotel: Pacific Grove Inn
nantucket - nearest hotel: Faraway Nantucket
shakhtersk - nearest hotel: No hotel found
tagta - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
65,dingli,MT,35.8614,14.3822,83,Maple farm B&B
123,mastung,PK,29.7994,66.8450,44,No hotel found
128,sirjan,IR,29.4520,55.6814,16,خانه معلم سیرجان
165,rodolfo sanchez taboada,MX,31.7167,-116.5667,49,Estero Beach Resort
180,gaigeturi,KR,33.4644,126.3183,78,제주 토비스콘도
248,kone,NC,-21.0595,164.8658,69,Pacifik Appartels
302,jamestown,US,42.0970,-79.2353,47,DoubleTree Jamestown
388,tripoli,LY,32.8752,13.1875,70,الاصيل
449,salmas,IR,38.1973,44.7653,31,هتل آتا
481,pacific grove,US,36.6177,-121.9166,72,Pacific Grove Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    tiles = "OSM",
    hover_cols = ["Lng","Lat", "City", "Humidity", "Hotel Name","Country"]
)


# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)